<a href="https://colab.research.google.com/github/xmendevs/ConsoleWars-Sales-Analysis-/blob/main/Paim_oil_Business_Investor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Palm Oil Profitability & Scenario Simulator ; for Nigerian German Business Man who wants to Invest in Agricultural Section of the econmy and in particular to generate a one Billion naira Profit every Year.


This notebook collects real public data (manual URLs), scrapes Jiji for land listings, extracts tables from company PDFs, cleans & merges real data, runs financial scenarios (Conservative / Base / Optimistic), runs NPV/IRR cashflow, and exports Power BI-ready outputs.

**States included:** Edo, Delta, Rivers, Akwa Ibom, Cross Rivers, Ondo




In [3]:
# Install required packages
!apt-get update -qq
!apt-get install -y -qq default-jdk
!apt-get install -y -qq chromium-browser
!pip install --upgrade pip
!pip install requests beautifulsoup4 pandas numpy tqdm tabula-py camelot-py[cv] selenium chromedriver-binary==114.0.5735.90 pdfminer.six openpyxl nbformat numpy_financial
print('Install complete - restart runtime if requested by Colab.')

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Install complete - restart runtime if requested by Colab.


In [4]:
import pandas as pd, os


os.makedirs("data_raw", exist_ok=True)

# Create market price scenarios file
price_df = pd.DataFrame({
    "Scenario": ["Conservative","Base","Optimistic"],
    "Price_per_t": [900000,1050000,1200000]  #manually inputed
})
price_df.to_csv("data_raw/market_prices.csv", index=False)

print("✅ market_prices.csv created successfully in /data_raw/")
price_df


✅ market_prices.csv created successfully in /data_raw/


,Scenario,Price_per_t
0,Conservative,900000
1,Base,1050000
2,Optimistic,1200000


In [5]:
state_data = [
    ["Edo",16.22,147147.79,20482600,812500,811500,2490],
    ["Akwa Ibom",10,147146.79,10480600,813500,812500,1871],
    ["Cross Rivers",8.99,147102,12482600,810500,812500,2251],
    ["Delta",9.69,147144.79,11482600,810100,805000,2487],
    ["Rivers",9,147142.79,15482600,811500,809500,2345],
    ["Ondo",7.9,147132.79,19482780,812000,807500,1409]
    ]

cols = ['State','Yield_t_ha','Cost_per_ha','LandCost_per_ha',
        'EstabCost_per_ha','Annual_OM_per_ha','Avg_Rainfall_mm']
state_df = pd.DataFrame(state_data, columns=cols)
state_df.to_csv("data_raw/state_factors.csv", index=False)

print("✅ state_factors.csv created successfully in /data_raw/")
state_df


✅ state_factors.csv created successfully in /data_raw/


,State,Yield_t_ha,Cost_per_ha,LandCost_per_ha,EstabCost_per_ha,Annual_OM_per_ha,Avg_Rainfall_mm
0,Edo,16.22,147147.79,20482600,812500,811500,2490
1,Akwa Ibom,10.00,147146.79,10480600,813500,812500,1871
2,Cross Rivers,8.99,147102.00,12482600,810500,812500,2251
3,Delta,9.69,147144.79,11482600,810100,805000,2487
4,Rivers,9.00,147142.79,15482600,811500,809500,2345
5,Ondo,7.90,147132.79,19482780,812000,807500,1409


## MODELLING PROPER

In [6]:
#  scenario modeling with maturity ramp-up
import math, os, pandas as pd, numpy as np
os.makedirs("outputs", exist_ok=True)

sf = 'data_raw/state_factors.csv'
mp = 'data_raw/market_prices.csv'
state_df = pd.read_csv(sf)
price_df = pd.read_csv(mp)

# Scenario multipliers
scenario_defaults = {
    'Conservative': {'yield_mult':0.90,'cost_mult':1.05},
    'Base':{'yield_mult':1.0,'cost_mult':1.0},
    'Optimistic':{'yield_mult':1.15,'cost_mult':0.95}
}

# Constants
TARGET_PROFIT = 1_000_000_000
AMORT_YEARS = 25
MATURE_YEAR = 4   # maturity begins in year 4 (i.e., years 0-3 are ramp)
MATURE_YIELD_DEFAULT = 4.0  # t/ha. Midpoint of 3-4 t/ha;

# Define ramp fractions for years  (fractions of mature yield)
# Year indexing: 0 = planting (no yield), Year1..Year3 = partial, Year4+ = full
ramp_by_year = {
    0: 0.0,
    1: 0.10,  # 10% of mature in year1
    2: 0.40,  # 40% of mature in year2
    3: 0.75,  # 75% of mature in year3
    # year >=4 will be 1.0 (mature)
}

rows = []
for _, p in price_df.iterrows():
    scen = p['Scenario']
    price_t = float(p['Price_per_t'])
    adj = scenario_defaults.get(scen, {'yield_mult':1.0,'cost_mult':1.0})
    for _, s in state_df.iterrows():
        state = s['State']
        # base yield from dataset (could be historical average) - keep but we will set mature_y
        base_y = float(s.get('Yield_t_ha', 0.0))
        # if user provided a Mature_Yield_t_ha column, use it; else use default logic
        if 'Mature_Yield_t_ha' in s and not pd.isna(s['Mature_Yield_t_ha']) and float(s['Mature_Yield_t_ha'])>0:
            mature_yield = float(s['Mature_Yield_t_ha'])
        else:
            # choose the larger of reported base yield and default mature yield
            mature_yield = max(base_y, MATURE_YIELD_DEFAULT)
        # apply scenario yield multiplier to mature yield (scenarios reflect agronomic +/-)
        mature_yield = mature_yield * adj['yield_mult']

        # annual recurring cost per ha (apply scenario cost multiplier)
        cost_ha = float(s.get('Cost_per_ha', 0.0)) * adj['cost_mult']
        annual_om = float(s.get('Annual_OM_per_ha', 0.0))
        estab = float(s.get('EstabCost_per_ha', 0.0))
        land = float(s.get('LandCost_per_ha', 0.0))

        # Steady-state revenue and costs per ha (mature)
        revenue_mature_per_ha = mature_yield * price_t
        total_costs_mature_per_ha = cost_ha + annual_om + (estab / AMORT_YEARS)
        profit_mature_per_ha = revenue_mature_per_ha - total_costs_mature_per_ha
        init_inv_per_ha = land + estab

        if profit_mature_per_ha > 0:
            hectares_needed = math.ceil(TARGET_PROFIT / profit_mature_per_ha)
            roi_annual = profit_mature_per_ha / init_inv_per_ha
            payback_years = init_inv_per_ha / profit_mature_per_ha
        else:
            hectares_needed = np.nan
            roi_annual = np.nan
            payback_years = np.nan

        # Add an entry with both mature and ramp-aware info
        rows.append({
            'State': state,
            'Scenario': scen,
            'Base_Yield_t_ha': base_y,
            'Mature_Yield_t_ha': mature_yield,
            'Price_per_t': price_t,
            'Revenue_mature_per_ha': revenue_mature_per_ha,
            'Total_Costs_mature_per_ha': total_costs_mature_per_ha,
            'Profit_mature_per_ha': profit_mature_per_ha,
            'Initial_Investment_per_ha': init_inv_per_ha,
            'Hectares_for_1B_at_mature': hectares_needed,
            'ROI_annual_at_mature': roi_annual,
            'Payback_years_at_mature': payback_years,
            'Avg_Rainfall_mm': s.get('Avg_Rainfall_mm', np.nan)
        })

results_mature = pd.DataFrame(rows)
results_mature.to_csv('outputs/scenario_results_mature.csv', index=False)
results_mature.to_excel('outputs/scenario_results_mature.xlsx', index=False)
print('✅ Saved scenario_results_mature.csv / .xlsx')
results_mature.head(12)


✅ Saved scenario_results_mature.csv / .xlsx


,State,Scenario,Base_Yield_t_ha,Mature_Yield_t_ha,Price_per_t,Revenue_mature_per_ha,Total_Costs_mature_per_ha,Profit_mature_per_ha,Initial_Investment_per_ha,Hectares_for_1B_at_mature,ROI_annual_at_mature,Payback_years_at_mature,Avg_Rainfall_mm
0,Edo,Conservative,16.22,14.598,900000.0,13138200.0,998505.1795,1.213969e+07,21295100.0,83,0.570070,1.754171,2490
1,Akwa Ibom,Conservative,10.00,9.000,900000.0,8100000.0,999544.1295,7.100456e+06,11294100.0,141,0.628687,1.590616,1871
2,Cross Rivers,Conservative,8.99,8.091,900000.0,7281900.0,999377.1000,6.282523e+06,13293100.0,160,0.472615,2.115886,2251
3,Delta,Conservative,9.69,8.721,900000.0,7848900.0,991906.0295,6.856994e+06,12292700.0,146,0.557810,1.792724,2487
4,Rivers,Conservative,9.00,8.100,900000.0,7290000.0,996459.9295,6.293540e+06,16294100.0,159,0.386247,2.589020,2345
5,Ondo,Conservative,7.90,7.110,900000.0,6399000.0,994469.4295,5.404531e+06,20294780.0,186,0.266302,3.755142,1409
6,Edo,Base,16.22,16.220,1050000.0,17031000.0,991147.7900,1.603985e+07,21295100.0,63,0.753218,1.327637,2490
7,Akwa Ibom,Base,10.00,10.000,1050000.0,10500000.0,992186.7900,9.507813e+06,11294100.0,106,0.841839,1.187876,1871
8,Cross Rivers,Base,8.99,8.990,1050000.0,9439500.0,992022.0000,8.447478e+06,13293100.0,119,0.635478,1.573618,2251
9,Delta,Base,9.69,9.690,1050000.0,10174500.0,984548.7900,9.189951e+06,12292700.0,109,0.747594,1.337624,2487


### NPV / IRR

In [7]:
# NPV / IRR with ramp-up to maturity
import numpy_financial as npf

DISCOUNT_RATE = 0.12
YEARS = 25

def cashflows_per_ha(state_row, price_t, adj):
    """
    state_row: a Series from state_df (original)
    price_t: price per tonne for the scenario
    adj: scenario multipliers dict with 'yield_mult' and 'cost_mult'
    returns: list of cashflows year0..YEARS-1
    """
    # determine mature yield
    base_y = float(state_row.get('Yield_t_ha', 0.0))
    if 'Mature_Yield_t_ha' in state_row and not pd.isna(state_row['Mature_Yield_t_ha']):
        mature_y = float(state_row['Mature_Yield_t_ha'])
    else:
        mature_y = max(base_y, MATURE_YIELD_DEFAULT)
    mature_y = mature_y * adj['yield_mult']

    cost_ha = float(state_row.get('Cost_per_ha', 0.0)) * adj['cost_mult']
    annual_om = float(state_row.get('Annual_OM_per_ha', 0.0))
    estab = float(state_row.get('EstabCost_per_ha', 0.0))
    land = float(state_row.get('LandCost_per_ha', 0.0))

    cfs = []
    for y in range(YEARS+1):  # year 0..YEARS inclusive for a 25-year plus initial
        if y == 0:
            # initial outflow: land + establishment in year 0
            cf = - (land + estab)
        else:
            # determine yield fraction for the year
            if y < MATURE_YEAR:
                frac = ramp_by_year.get(y, 0.0)
            else:
                frac = 1.0
            yield_t = mature_y * frac
            revenue = yield_t * price_t
            costs = cost_ha + annual_om
            cf = revenue - costs
        cfs.append(cf)
    return cfs

# compute NPV and IRR for each state and scenario
npv_rows = []
for _, p in price_df.iterrows():
    scen = p['Scenario']
    price_t = float(p['Price_per_t'])
    adj = scenario_defaults.get(scen, {'yield_mult':1.0,'cost_mult':1.0})
    for _, s in state_df.iterrows():
        cfs = cashflows_per_ha(s, price_t, adj)
        # discount NPV
        npv = sum([cf / ((1 + DISCOUNT_RATE) ** t) for t, cf in enumerate(cfs)])
        # compute IRR (numpy_financial.irr expects list of cashflows)
        try:
            irr = npf.irr(cfs)
        except Exception:
            irr = np.nan
        npv_rows.append({
            'State': s['State'],
            'Scenario': scen,
            'NPV_per_ha': npv,
            'IRR_per_ha': irr,
            'CF_year_0': cfs[0],
            'CF_year_1': cfs[1] if len(cfs)>1 else None,
            'CF_mature_year': cfs[MATURE_YEAR] if len(cfs)>MATURE_YEAR else None
        })

npv_df = pd.DataFrame(npv_rows)
npv_df.to_csv('outputs/npv_irr_by_state_mature.csv', index=False)
print('✅ Saved outputs/npv_irr_by_state_mature.csv')
npv_df.head(12)


✅ Saved outputs/npv_irr_by_state_mature.csv


,State,Scenario,NPV_per_ha,IRR_per_ha,CF_year_0,CF_year_1,CF_mature_year
0,Edo,Conservative,5.499354e+07,0.338901,-21295100.0,347814.8205,1.217219e+07
1,Akwa Ibom,Conservative,3.282633e+07,0.353884,-11294100.0,-157004.1295,7.132996e+06
2,Cross Rivers,Conservative,2.560552e+07,0.288317,-13293100.0,-238767.1000,6.314943e+06
3,Delta,Conservative,3.028372e+07,0.325539,-12292700.0,-174612.0295,6.889398e+06
4,Rivers,Conservative,2.267942e+07,0.249241,-16294100.0,-234999.9295,6.326000e+06
5,Ondo,Conservative,1.300698e+07,0.185861,-20294780.0,-322089.4295,5.437011e+06
6,Edo,Base,7.990018e+07,0.414840,-21295100.0,744452.2100,1.607235e+07
7,Akwa Ibom,Base,4.820397e+07,0.437590,-11294100.0,90353.2100,9.540353e+06
8,Cross Rivers,Base,3.943583e+07,0.360081,-13293100.0,-15652.0000,8.479898e+06
9,Delta,Base,4.518645e+07,0.403825,-12292700.0,65305.2100,9.222355e+06


# NEXT LINE OF ACTION FOR THIS PROJECT


next is to Create What-If parameters: `YieldAdjustmentPct`, `PriceAdjustmentPct`, `CostAdjustmentPct` (-20 to 20). Then create the DAX measures shown in the notebook for Adjusted_Yield, Adjusted_Price, Adjusted_Cost, Adjusted_Revenue, Adjusted_Profit, Adjusted_ROI, and Hectares_for_1B.
